In [ ]:
import json
import os
import sys
import fnmatch


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('BBGM_League_2_all_seasons_Average_Stats.csv')

In [ ]:
y = df.iloc[:,-15:]
X = df.iloc[:,11:-17]
y = y[(X['AST%'] >0) & (X['AST%'] < 100)]
X = X[(X['AST%'] >0) & (X['AST%'] < 100)]
X['MP'] = df.MP
X['Hgt'] = df['Hgt']
df.columns


In [ ]:
stat_list = ['FG','FGA','3P',"3PA",'FT','FTA',\
             'ORB','DRB','TRB','AST','TOV','STL',"Blk",\
             'PF','PTS']
for name in stat_list:
    den = np.maximum(1,df.MP)
    #if name in ['OWS','DWS']:
    #    den = den*df['G']
    #X[name] = df[name]
    X[name + 'p36'] = 36* df[name]/den
    X[name + 'p100'] = X[name + 'p36']*4/3

X['3PtP'] = (2/(1+np.exp(-X['3PAp100']))-1)*X['3P%']/100
X['Creation'] = X['ASTp100']*0.1843+(X['PTSp100']+X['TOVp100'])*0.0969-2.3021*X['3PtP']+0.0582*(X['ASTp100']*(X['PTSp100']+X['TOVp100'])*X['3PtP'] )-1.1942
X['Load'] = (X['ASTp100']-(0.38*X['Creation'])*0.75)+X['FGAp100']+X['FTAp100']*0.44+X['Creation']+X['TOVp100']
X['cTOV'] = X['TOVp100']/X['Load']
X['DPM'] = X['Blkp100']*0.802+X['DRBp100']*0.42-4.7-0.07551*X['PFp100']+1.597019*X['STLp100']-0.26385*X['TOVp100']
X['OPM'] = -8.57647+0.6111*X['PTSp100']-0.33918*(0.44*X['FTAp100']+X['FGAp100'])+0.440814*X['FTAp100']+0.379745*X['3PAp100']+0.634044*X['ASTp100']+0.77827*X['ORBp100']-1.08855*X['TOVp100']+0.26262*X['STLp100']
X['BPM'] = X['OPM'] + X['DPM']
X['Age'] = df['Age']

#X['PassP'] = ((X['ASTp100']-(0.38*X['Creation']))*0.752+ X['Creation'] + X['TOVp100']) ** 0.67
#'OPM','DPM','cTOV','Load'#stat_list[:-2]+
X = X[[_ for _ in X.columns if '%' in _ or _[-1]=='r' or 'FGP' in _ or "+/-" in _ or 'p36' in _ or _ in (['Creation','cTOV','Load','Age','MP'])]]

replacement_filter = (df.Salary > 0.5) & (df.Salary < 1.0)
replacement_player_mean_bs = X[replacement_filter].mean()
replacement_player_std_bs = X[replacement_filter].std()
replacement_player_cov_bs = X[replacement_filter].cov()

replacement_player_mean_r = y[replacement_filter].mean()
replacement_player_std_r = y[replacement_filter].std()
replacement_player_cov_r = y[replacement_filter].cov()


replacement_player_mean_r

X.columns


In [ ]:
#_ =  plt.hist(X['OWSp36'],150)

In [ ]:
from sklearn import neural_network
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import feature_selection
from sklearn import  multioutput
from sklearn import ensemble
from sklearn import svm

In [ ]:
fexp = preprocessing.PolynomialFeatures(degree=2,interaction_only=True)
scalerX = preprocessing.RobustScaler()
scalery = preprocessing.StandardScaler()
prescale_X = scalerX.fit_transform(X)
prescale_y = scalery.fit_transform(y)
prescale_X = fexp.fit_transform(prescale_X)

In [ ]:
trials = 1
ts = []
for i in range(trials):
    #clf = neural_network.MLPRegressor((36,5,24,36),'tanh',solver='adam',max_iter=1000)
    clf = neural_network.MLPRegressor((),'identity',solver='lbfgs',alpha=3e2)
    #clf = multioutput.MultiOutputRegressor(linear_model.SGDRegressor(penalty='l1',alpha=1e-3,tol=1e-3,max_iter=50))
    #clf = ensemble.ExtraTreesRegressor(8,criterion='mae',max_depth=8,verbose=1)
    #clf = multioutput.MultiOutputRegressor(svm.SVR())
    clf.fit(prescale_X,prescale_y)
    yt = scalery.inverse_transform(clf.predict(prescale_X))
    err = np.linalg.norm(yt-y)
    ts.append((err,clf))

In [ ]:
ts = sorted(ts)[::1] # why not the biggest error
print(ts[0][0])
clf = ts[0][1]

In [ ]:
col_names = X.columns
col_names = fexp.get_feature_names(X.columns)

for i,c in enumerate(y.columns):
    coeffs = clf.coefs_[-1][:,i]
    v = np.argsort(abs(coeffs))[::-1]
    print(c)
    coeffs2 = [(coeffs[i2],col_names[i2]) for i2 in v[:5]]
    #for v,n in sorted(coeffs2,reverse=True):
    #    print('{:.2f} * {} + '.format(v,n),end='')
    print('| Variable | Coeff |')
    print('|----------|-------|')
    for v,n in sorted(coeffs2,reverse=True):
        print('|{:25s}|{:.2f}|'.format(n,v))
    #for v,n in sorted(coeffs2,reverse=True):
    #    print('\t{:25s}\t{:.2f}'.format(n,v))


In [ ]:
GEN_YEAR = 2019
tyear = [GEN_YEAR]
if tyear[0] < 1980:
    print("MY PARSING OF THE TABLES IS WRONG WITHOUT the 2PA/3PA TRACKS")
    raise

CURRENT_YEAR = 2019
if tyear[0] <= CURRENT_YEAR-1:
    # fill out the next 3 drafts
    tyear.append(tyear[0]+1)
if tyear[0] <= CURRENT_YEAR-2:
# fill out the next 3 drafts
    tyear.append(tyear[0]+2)
if tyear[0] <= CURRENT_YEAR-3:
    # fill out the next 3 drafts
    tyear.append(tyear[0]+3)
if GEN_YEAR >= 1981:
    tyear.append(GEN_YEAR-1)
all_tables = {}
for ty in tyear: 
    all_tables[ty] = np.load('tables_{}.pkl'.format(ty))
teams = all_tables[tyear[0]].keys()


In [ ]:
player_stats = {k:{} for k in tyear}
table_columns = {}
for ty in tyear:
    tables = all_tables[ty]
    for team in tables:
        team_tables =  tables[team]
        for table_name in team_tables:
            if table_name in ['draft-rights','team_and_opponent']:
                continue
            table = team_tables[table_name].fillna(0)
            #print(table_name)
            #print(table.index)
            for row in table.itertuples():
                name = row[0]
                name = name.replace('\xa0\xa0',' ')
                if name == 'Team Totals':
                    continue
                nsplit = name.split(' ')
                if nsplit[-1] in ['Jr.','Sr.','I','II','III',"IV",'(TW)']:
                    name = ' '.join(nsplit[:-1])

                player_table = player_stats[ty].get(name,{})
                player_row = player_table.get(table_name,[])
                player_row = player_row + [row]

                player_table[table_name] = player_row
                player_stats[ty][name] = player_table
                #if name == 'Dennis Smith Jr.' or name == 'Luka Doncic':
                #    print(player_stats[ty][name],team)
            table_columns[table_name] = table.columns


In [ ]:
table_mask = {}
for table in table_columns:
    table_mask[table] = [_.strip() !=''  for _ in table_columns[table] ]
    table_columns[table] = [_ for _ in table_columns[table] if _.strip() != '']
#for player in player_stats:
#    for table_in in player_stats[player]:
#        if 'on_off' in table_in or 'salaries' in table_in:
#            continue
#        if len(player_stats[player][table_in]) > 1:
#            pass
            #print(player,table_in,'MP' in player_stats[player][table_in][0]._fields)
            #print(player_stats[player][table_in][0])

In [ ]:
# add playoff data to normal data
if True:
    for ty in tyear:
        for player in player_stats[ty]:
            for table_in in player_stats[ty][player]:
                tableN = table_in.split('_')
                tableS = '_'.join(tableN[1:])
                if 'playoffs'==tableN[0] and not table_in in ['playoffs_pbp']:
                    #print(table_in)
                    if tableS in player_stats[ty][player]:
                        player_stats[ty][player][tableS] += player_stats[ty][player][table_in]

In [ ]:
for ty in tyear:
    for player in player_stats[ty]:
        for tt in player_stats[ty][player]:
            new_rows = []
            for tablet in player_stats[ty][player][tt]:
                vector = [_ if _ != '' else '0.5' for _ in tablet[1:]]
                vector = [(float(_.replace('%',''))/100 if type(_) == str and'%' in _ else _) for _ in vector]
                if 'on_off' in tt:
                    vector = vector[1:]
                if 'contracts' in tt:
                    vector = vector[1:-2]
                if tt in ['salaries2','contracts']:
                    vector = [_.replace(',','').replace('$','') for _ in vector]
                try:
                    v2 = np.array(vector).astype(np.float)
                except:
                    v2 = vector
                new_rows.append(vector)
            a = np.array(new_rows)

            if 'MP' in table_columns[tt] and not tt in ['pbp','on_off','on_off_p']:
                a = a.astype(np.float)
                try:
                    mins = a[:,table_columns[tt].index('MP')].reshape((-1,1))
                    new_rows = ((a.T @ mins)/mins.sum()).T
                    a = new_rows
                except:
                    print(tt,a.shape,player,a,mins)
                    raise

            player_stats[ty][player][tt] = a


In [ ]:
#player_stats['Ben Simmons'].keys()

In [ ]:
player_vectors = []
player_names = []
player_years = []
player_scales = []

for ty in tyear:
    for name in player_stats[ty]:
        #name = 'Jimmy Butler'
        #name = 'Ben Simmons'
        stats = player_stats[ty][name]
        if not 'totals' in stats:
            if ty == GEN_YEAR and name in player_stats[tyear[-1]] and 'totals' in player_stats[tyear[-1]][name]:
                stats = player_stats[tyear[-1]][name]
                print(name,ty,tyear)
            else:
                continue
        player_names.append(name)
        player_years.append(ty)
        try:
            ht = [int(_) for _ in stats['roster'][0][1].split('-')]
            hgt = ht[0]*12 + ht[1]
        except:
            hgt=70
        if 'shooting' in stats:
            d = {
            'AtRimFGP':stats['shooting'][0][11]*100,
            'LowPostFGP':stats['shooting'][0][11]*100, #12
            'MidRangeFGP':stats['shooting'][0][13]*100}
        else:
            d = {
            'AtRimFGP':stats['totals'][0][12]*100,
            'LowPostFGP':stats['totals'][0][12]*100, #12
            'MidRangeFGP':stats['totals'][0][13]*100}
        if 'roster' in stats:
            d['Age']= GEN_YEAR-int(stats['roster'][0][3].split(',')[-1])
        else:
            d['Age']=np.random.randint(25,35)
            #print(name)

        #for stat in X.columns:
        d.update({'FG%':stats['totals'][0][6]*100,
            'Hgt':3.7*(hgt-66),
            'FG': stats['per_game'][0][4],
            'FGA': stats['per_game'][0][5],
            '3P': stats['per_game'][0][7],
            '3PA': stats['per_game'][0][8],
            'FT': stats['per_game'][0][14],
            'FTA': stats['per_game'][0][15],
            'ORB': stats['per_game'][0][17],
            'DRB': stats['per_game'][0][18],
            'TRB': stats['per_game'][0][19],
            'AST': stats['per_game'][0][20],
            'STL': stats['per_game'][0][21],
            'Blk': stats['per_game'][0][22],
            'TOV': stats['per_game'][0][23],
            'PF': stats['per_game'][0][24],
            'PTS': stats['per_game'][0][25],
            'OWS':stats['advanced'][0][16],
            'DWS':stats['advanced'][0][17],
            '3P%':stats['totals'][0][9]*100,
            'MP':stats['per_game'][0][3],
            'FT%':stats['totals'][0][16]*100,
            'TS%':stats['advanced'][0][4]*100,
            '3PAr':stats['advanced'][0][5]*100,
            'FTr':stats['advanced'][0][6]*100,
            'ORB%':stats['advanced'][0][7],
            'DRB%':stats['advanced'][0][8],
            'TRB%':stats['advanced'][0][9],
            'AST%':stats['advanced'][0][10],
            'STL%':stats['advanced'][0][11],
            'BLK%':stats['advanced'][0][12],
            'TOV%':stats['advanced'][0][13],
            'USG%':stats['advanced'][0][14],
            '+/-':stats['advanced'][0][22],
            'FGp36':stats['per_minute'][0][4],
            'FGAp36':stats['per_minute'][0][5],
            '3Pp36':stats['per_minute'][0][7],
            '3PAp36':stats['per_minute'][0][8],
            'FTp36':stats['per_minute'][0][13],
            'FTAp36':stats['per_minute'][0][14],
            'ORBp36':stats['per_minute'][0][16],
            'DRBp36':stats['per_minute'][0][17],
            'TRBp36':stats['per_minute'][0][18],
            'ASTp36':stats['per_minute'][0][19],
            'TOVp36':stats['per_minute'][0][22],
            'STLp36':stats['per_minute'][0][20],
            'Blkp36':stats['per_minute'][0][21],
            'PFp36':stats['per_minute'][0][23],
            'PTSp36':stats['per_minute'][0][24],
            'FGp100':stats['per_poss'][0][4],
            'FGAp100':stats['per_poss'][0][5],
            '3Pp100':stats['per_poss'][0][6],
            '3PAp100':stats['per_poss'][0][7],
            'FTp100':stats['per_poss'][0][12],
            'FTAp100':stats['per_poss'][0][13],
            'ORBp100':stats['per_poss'][0][16],
            'DRBp100':stats['per_poss'][0][17],
            'TRBp100':stats['per_poss'][0][18],
            'ASTp100':stats['per_poss'][0][19],
            'TOVp100':stats['per_poss'][0][22],
            'STLp100':stats['per_poss'][0][20],
            'Blkp100':stats['per_poss'][0][21],
            'PFp100':stats['per_poss'][0][23],
            'PTSp100':stats['per_poss'][0][24],
            'PER':stats['advanced'][0][3],
            'DPM':stats['advanced'][0][17],
            'OPM':stats['advanced'][0][16],
            'BPM':stats['advanced'][0][18],

            'OWSp36':36*stats['advanced'][0][16]/stats['per_game'][0][3],
            'DWSp36':36*stats['advanced'][0][17]/stats['per_game'][0][3]})
        d['3PtP'] = (2/(1+np.exp(-d['3PAp100']))-1)*d['3P%']/100
        d['Creation'] = d['ASTp100']*0.1843+(d['PTSp100']+d['TOVp100'])*0.0969-2.3021*d['3PtP']+0.0582*(d['ASTp100']*(d['PTSp100']+d['TOVp100'])*d['3PtP'] )-1.1942
        d['Load'] = (d['ASTp100']-(0.38*d['Creation'])*0.75)+d['FGAp100']+d['FTAp100']*0.44+d['Creation']+d['TOVp100']
        d['cTOV'] = d['TOVp100']/d['Load']
        d['DPM'] = d['Blkp100']*0.802+d['DRBp100']*0.42-4.7-0.07551*d['PFp100']+1.597019*d['STLp100']-0.26385*d['TOVp100']
        d['OPM'] = -8.57647+0.6111*d['PTSp100']-0.33918*(0.44*d['FTAp100']+d['FGAp100'])+0.440814*d['FTAp100']+0.379745*d['3PAp100']+0.634044*d['ASTp100']+0.77827*d['ORBp100']-1.08855*d['TOVp100']+0.26262*d['STLp100']
        d['BPM'] = d['OPM']+d['DPM']

        MP = np.nan_to_num(stats['totals'][0][3])
        #if np.isnan(mp_scale):
        #    print(name,ty,MP)
        player_scales.append(MP)
        #d['PassP'] = ((d['ASTp100']-(0.38*d['Creation']))*0.752+ d['Creation'] + d['TOVp100']) ** 0.67
        player_vectors.append([d[stat] for stat in X.columns])
X.columns

In [ ]:
#stat_list
#player_stats[2019]['Stephen Curry']['per_minute'][0][14]

In [ ]:
first_n = len([yr for yr in player_years if yr == tyear[0]])
gen_FA = len(teams)*5 if CURRENT_YEAR!=GEN_YEAR else 0
first_n,len(teams),gen_FA


In [ ]:
Xn = np.nan_to_num(np.array(player_vectors))
scalerX2 = preprocessing.StandardScaler()
scalerX2.fit(Xn[:first_n])
#scalerX2 = scalerX
Xn_s =scalerX2.transform(np.nan_to_num(Xn))
Xn_fs = fexp.transform(np.nan_to_num(Xn_s))
predict = clf.predict(Xn_fs)
ratings = np.nan_to_num(scalery.inverse_transform(predict))
#ratings[:,0] = Xn[:,list(X.columns).index('Hgt')]
HGT_PRED = 0.75
if 'Hgt' in list(X.columns):
    ratings[:,0] = HGT_PRED*ratings[:,0] + (1-HGT_PRED)* Xn[:,list(X.columns).index('Hgt')]
# now we can sample directly from ratings

# if we want to scale players down based on minutes played to replacement level
if False:
    MEAN_S = 0.95
    STD_S = 0.25

    c = np.array(player_scales).reshape((-1,1))
    c = np.tanh(c/600) # basically 1 by 1200 MP

    ratings[:Xn.shape[0]] = ratings[:Xn.shape[0]]*c + (1-c)*np.repeat(np.array(replacement_player_mean_r).reshape((-1,1)),Xn.shape[0],1).T


In [ ]:
#table_columns['per_minute']#,table_columns['advanced'].index('0-3')
#for i,t in enumerate(player_stats[name]['advanced'][0]):
#    print(i,t)

In [ ]:
#player_vectors[player_names.index('Joel Embiid')][list(X.columns).index('OWSp36')],X.columns[list(X.columns).index('OWSp36')]
#player_stats[2019]['Joel Embiid']['advanced'][0][12],player_stats[2019]['Joel Embiid']['per_game'][0][3]

In [ ]:
_ = plt.hist(Xn[:,list(X.columns).index('FTp36')],150,density=True,alpha=0.5,label='NBA')
_ = plt.hist(X['FTp36'],150,alpha=0.5,density=True,label='BBGM')
plt.legend()

In [ ]:
X.mean(0)-Xn.mean(0)

In [ ]:
for n,v in zip(X.columns,Xn.max(0)):
    print(n,v)

In [ ]:
rosters = {}

for team in all_tables[tyear[0]]:
    for name in all_tables[tyear[0]][team]['roster'].index:
        nsplit = name.split(' ')
        if nsplit[-1] in ['Jr.','Sr.','I','II','III',"IV",'(TW)']:
            name = ' '.join(nsplit[:-1])
        rosters[name] = team
    #print(team,)


In [ ]:
def load_roster(filename):
    base = json.load(open(filename,'rb'))

    
    for i in range(len(base['players'])):
        player = base['players'][i]
        if 'name' in player and player['name'] !='':
            sname = base['players'][i]['name'].split()
            base['players'][i]['firstName'] = sname[0]
            base['players'][i]['lastName'] = ' '.join(sname[1:])
            del base['players'][i]['name']
    players = base['players']
    base['version'] = 32

    return players,base
playersOG,base = load_roster('normed.json')
abbrev_to_tid = {_['abbrev']:_['tid']for _ in base['teams']}
abbrev_to_tid['PHO'] = abbrev_to_tid['PHX']
abbrev_to_tid['CHO'] = abbrev_to_tid['CHA']
abbrev_to_tid['BRK'] = abbrev_to_tid['BKN']
abbrev_to_tid['NOH'] = abbrev_to_tid['NOP']
abbrev_to_tid['NJN'] = abbrev_to_tid['BKN']
abbrev_to_tid['SEA'] = abbrev_to_tid['OKC']
abbrev_to_tid['KCK'] = abbrev_to_tid['SAC']
abbrev_to_tid['SDC'] = abbrev_to_tid['LAC']
abbrev_to_tid['WSB'] = abbrev_to_tid['WAS']
abbrev_to_tid['CHH'] = abbrev_to_tid['CHA']

base['startingSeason'] = tyear[0]
valid_tids = {abbrev_to_tid[_] for _ in teams}

valid_teams = []
for t in base['teams']:
    if t['tid'] in valid_tids:
        tid = len(valid_teams)
        otid = t['tid']
        t['tid'] = tid
        for k,v in abbrev_to_tid.items():
            if v == otid:
                abbrev_to_tid[k] = tid
        valid_teams.append(t)
base['teams'] = valid_teams
if tyear[0] < CURRENT_YEAR-2:
    del base['draftPicks']

players_discard = [_ for _ in playersOG if  _['tid'] > -3 ]
players_FA = [_ for _ in playersOG if  _['tid'] == -1 ]

players_to_keep = []#[-1,-2,-4,-5]

#-2 is for undrafted players the ongoing or upcoming draft
#-4 is for prospects in next year's draft
#-5 is for prospects in the draft after that.
players = []
if GEN_YEAR == CURRENT_YEAR:
    players += [_ for _ in playersOG if _['tid'] in [-1,-2,-4,-5]]
elif GEN_YEAR == CURRENT_YEAR-1:
    for player in playersOG:
        if player['tid'] == -2:
            player['tid'] = -4
            players.append(player)
        elif player['tid'] == -4:
            player['tid'] = -5
            players.append(player)
elif tyear[0] == CURRENT_YEAR-2:
    for player in playersOG:
        if player['tid'] == -2:
            player['tid'] = -5
            players.append(player)

y_keys = [_.lower() for _ in y.columns]

y_map = { 'hgt': 'hgt',
   'stre': 'str',
   'spd': 'spd',
   'jmp': 'jmp',
   'endu': 'end',
   'ins': 'ins',
   'dnk': 'dnk',
   'ft': 'ft.1',
   'fg': '2pt',
   'tp': '3pt',
   'diq': 'diq',
   'oiq': 'oiq',
   'drb': 'drb',
   'pss': 'pss',
   'reb': 'reb' }

In [ ]:
if False:
    ri = 0
    _ = plt.hist(yt[:,ri],100,normed=True,alpha=0.5,label='predicted')
    _ = plt.hist(y.iloc[:,ri],100,normed=True,alpha=0.5,label='labels')
    _ = plt.hist(ratings[:,ri],100,normed=True,alpha=0.5,label='dataset')
    plt.xlim(-100,100)
    plt.legend()
elif False:
    plt.figure(figsize=(7*3,6*3))

    for ri in range(len(X.columns)):
        #ri = 13
        #plt.figure()
        plt.subplot(6,7,ri+1)

        _ = plt.hist(Xn[:,ri],100,density=True,alpha=0.5,label='NBA')
        _ = plt.hist(X.iloc[:,ri],100,density=True,alpha=0.5,label='BBGM')
        plt.legend()
        plt.title(X.columns[ri])# + ' ' + str(ri))
        plt.tight_layout()
        #plt.xlim(-100,100)
from collections import OrderedDict

def write_roman(num):

    roman = OrderedDict()
    roman[1000] = "M"
    roman[900] = "CM"
    roman[500] = "D"
    roman[400] = "CD"
    roman[100] = "C"
    roman[90] = "XC"
    roman[50] = "L"
    roman[40] = "XL"
    roman[10] = "X"
    roman[9] = "IX"
    roman[5] = "V"
    roman[4] = "IV"
    roman[1] = "I"

    def roman_num(num):
        for r in roman.keys():
            x, y = divmod(num, r)
            yield roman[r] * x
            num -= (r * x)
            if num <= 0:
                break

    return "".join([a for a in roman_num(num)])

In [ ]:
if gen_FA > 0:
    player_names_f = player_names + ["Free Agent{}".format(write_roman(i)) for i in range(gen_FA)]
    player_years_f = player_years + [GEN_YEAR for i in range(gen_FA)]
    # scale them down, we don't want a bunch of amazing replacements
    MEAN_S = 0.95
    STD_S = 0.25
    rp_ratings = np.random.multivariate_normal(MEAN_S*replacement_player_mean_r,STD_S*replacement_player_cov_r,size=(gen_FA))
    ratings_f = np.vstack([ratings,rp_ratings])
else:
    player_names_f = player_names
    player_years_f = player_years
    ratings_f = ratings

In [ ]:
import pprint
import copy

pp = pprint.PrettyPrinter()
for i,name in enumerate(player_names_f):
    py = player_years_f[i]
    name = player_names_f[i]
    sname = name.split(' ')
    new_player = {}
    new_player['firstName'] = sname[0]
    new_player['lastName'] = ' '.join(sname[1:])
    year_gap = 0
    if name in player_stats[py] and 'roster' in player_stats[py][name]:
        #if name == 'Ben Simmons':
        #    print(name,py)
        exp_years =  player_stats[py][name]['roster'][0][5]
        exp_years = int(0 if exp_years == 'R' else exp_years)

        year_gap = (py-GEN_YEAR) #0,1,2


        if py == GEN_YEAR and name in rosters:
            tid = abbrev_to_tid[rosters[name]]
        elif exp_years != 0:
            continue
        elif py < GEN_YEAR:
            continue
        else:
            tid = [-2,-4,-5][year_gap-1]

        new_player['tid'] = tid
        new_player['pos'] = player_stats[py][name]['roster'][0][0]
        new_player['weight'] = int(player_stats[py][name]['roster'][0][2])
        ht = [int(_) for _ in player_stats[py][name]['roster'][0][1].split('-')]
        new_player['hgt'] = ht[0]*12 + ht[1]
        #if name == 'Sleepy Floyd':
        #    print(ht,new_player)
        #new_player['draft'] = {'year':py-exp_years}

        new_player['born'] = {'year':int(player_stats[py][name]['roster'][0][3].split(',')[-1]),'loc':''}
        if tyear[0]>=CURRENT_YEAR and 'contracts' in player_stats[py][name]:
            #print(name,player_stats[py][name]['contracts'])
            ctrct = [float(_) for _ in player_stats[py][name]['contracts'][0] if _.replace('\xa0','0.5') != '0.5']
            if len(ctrct) > 0:
                new_player['contract'] = {'amount':int(round(sum(ctrct)/(1000*len(ctrct)))),'exp':CURRENT_YEAR-1+len(ctrct)}
    elif i > Xn.shape[0]:
        print(name)
        new_player['tid'] = int(-1)
        new_player['weight'] = int(np.random.normal(220,20))
        new_player['hgt'] = int(np.random.normal(6*12+6,3))
        new_player['born'] = {'year':int(GEN_YEAR-np.random.normal(32,2)),'loc':''}
        new_player['pos'] = "GF"
        #print(new_player)
    else:
        #print(name)
        continue
    #if name == 'Ben Simmons':
    #    print(new_player)
    #print(year_gap)
    scale_rookie = [1.0,1.0,1.0,1.0][year_gap]
    new_player['ratings'] = [{k: int(scale_rookie*np.clip(ratings_f[i,y_keys.index(km)],0,100)) for k,km in y_map.items()}]
    
    players.append(new_player)
    #if name in ['Luka Doncic']: #"Trevon Duval",'LeBron James'
        #print(py,exp_years)
    #    pp.pprint(new_player)
        #pp.pprint(player_vectors[player_names.index(name)])

In [ ]:
base['players'] = players
with open('auto_roster_{}.json'.format(tyear[0]),'wt') as fp:
    json.dump(base,fp, indent=4, sort_keys=True)

In [ ]:
len(ratings),len(ratings_f),len(ratings)+gen_FA